# Casava Leaf Disease Classification with ResNet

In [ ]:
import pandas  as pd
import numpy as np
import matplotlib.pyplot  as plt
from sklearn.utils import shuffle
import cv2

import tensorflow as tf 
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, GlobalAveragePooling2D

**Define Dataset path**

In [ ]:
data_path = "../input/cassava-leaf-disease-classification/"
train_csv_data_path = data_path+"train.csv"
label_json_data_path = data_path+"label_num_to_disease_map.json"
images_dir_data_path = data_path+"train_images"

In [ ]:
train_csv = pd.read_csv(train_csv_data_path)
train_csv['label'] = train_csv['label'].astype('string')

label_class = pd.read_json(label_json_data_path, orient='index')
label_class = label_class.values.flatten().tolist()

In [ ]:
print("Label names :")
for i, label in enumerate(label_class):
    print(f" {i}. {label}")

In [ ]:
train_csv.head()

In [ ]:

# Data agumentation and pre-processing using Keras
train_gen = ImageDataGenerator(
                                rotation_range=360,
                                width_shift_range=0.1,
                                height_shift_range=0.1,
                                brightness_range=[0.1,0.9],
                                shear_range=25,
                                zoom_range=0.3,
                                channel_shift_range=0.1,
                                horizontal_flip=True,
                                vertical_flip=True,
                                rescale=1/255,
                                validation_split=0.15
                               )
                                    
    
valid_gen = ImageDataGenerator(rescale=1/255,
                               validation_split = 0.15
                              )



In [ ]:
BATCH_SIZE = 18
IMG_SIZE = 224

In [ ]:
train_generator = train_gen.flow_from_dataframe(
                            dataframe=train_csv,
                            directory = images_dir_data_path,
                            x_col = "image_id",
                            y_col = "label",
                            target_size = (IMG_SIZE, IMG_SIZE),
                            class_mode = "categorical",
                            batch_size = BATCH_SIZE,
                            shuffle = True,
                            subset = "training",

)

valid_generator = valid_gen.flow_from_dataframe(
                            dataframe=train_csv,
                            directory = images_dir_data_path,
                            x_col = "image_id",
                            y_col = "label",
                            target_size = (IMG_SIZE, IMG_SIZE),
                            class_mode = "categorical",
                            batch_size = BATCH_SIZE,
                            shuffle = False,
                            subset = "validation"
)

In [ ]:
batch = next(train_generator)
images = batch[0]
labels = batch[1]

In [ ]:
plt.figure(figsize=(12,9))
for i, (img, label) in enumerate(zip(images, labels)):
    plt.subplot(2,3, i%6 +1)
    plt.axis('off')
    plt.imshow(img)
    plt.title(label_class[np.argmax(label)])
    
    if i==15:
        break

# Building The Model

In [ ]:
# Loading the ResNet152 architecture with imagenet weights as base
base = tf.keras.applications.ResNet152(include_top=False, weights='imagenet',input_shape=[IMG_SIZE,IMG_SIZE,3])

In [ ]:
base.summary()

In [ ]:
model = tf.keras.Sequential()
model.add(base)
model.add(BatchNormalization(axis=-1))
model.add(GlobalAveragePooling2D())
model.add(Dense(5, activation='softmax'))

In [ ]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=tf.keras.optimizers.Adamax(learning_rate=0.01), metrics=['acc'])

In [ ]:
model.summary()

# Training The Model

Loading the saved model


In [ ]:
history = model.fit(
      train_generator,
      steps_per_epoch=BATCH_SIZE,
      epochs=20,
      validation_data=valid_generator,
      batch_size=BATCH_SIZE
      )

In [ ]:
model.save('ResNet152.h5')

In [ ]:
# Loading the ResNet101 architecture with imagenet weights as base
base = tf.keras.applications.ResNet101(include_top=False, weights='imagenet',input_shape=[IMG_SIZE,IMG_SIZE,3])

model = tf.keras.Sequential()
model.add(base)
model.add(BatchNormalization(axis=-1))
model.add(GlobalAveragePooling2D())
model.add(Dense(5, activation='softmax'))

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=tf.keras.optimizers.Adamax(learning_rate=0.01), metrics=['acc'])

history = model.fit(
      train_generator,
      steps_per_epoch=BATCH_SIZE,
      epochs=20,
      validation_data=valid_generator,
      batch_size=BATCH_SIZE
      )

model.save('ResNet101.h5')

In [ ]:
# Loading the ResNet50 architecture with imagenet weights as base
base = tf.keras.applications.ResNet50(include_top=False, weights='imagenet',input_shape=[IMG_SIZE,IMG_SIZE,3])

model = tf.keras.Sequential()
model.add(base)
model.add(BatchNormalization(axis=-1))
model.add(GlobalAveragePooling2D())
model.add(Dense(5, activation='softmax'))

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=tf.keras.optimizers.Adamax(learning_rate=0.01), metrics=['acc'])

history = model.fit(
      train_generator,
      steps_per_epoch=BATCH_SIZE,
      epochs=20,
      validation_data=valid_generator,
      batch_size=BATCH_SIZE
      )

model.save('ResNet50.h5')

In [ ]:
model = tf.keras.models.load_model("/kaggle/working/ResNet50.h5")

In [ ]:
test_img_path = data_path+"test_images/2216849948.jpg"

img = cv2.imread(test_img_path)
resized_img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)).reshape(-1, IMG_SIZE, IMG_SIZE, 3)/255

plt.figure(figsize=(8,4))
plt.title("TEST IMAGE")
plt.imshow(resized_img[0])

In [ ]:
preds = []
ss = pd.read_csv(data_path+'sample_submission.csv')

for image in ss.image_id:
    img = tf.keras.preprocessing.image.load_img(data_path+'test_images/' + image)
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = tf.keras.preprocessing.image.smart_resize(img, (IMG_SIZE, IMG_SIZE))
    img = tf.reshape(img, (-1, IMG_SIZE, IMG_SIZE, 3))
    prediction = model.predict(img/255)
    preds.append(np.argmax(prediction))

my_submission = pd.DataFrame({'image_id': ss.image_id, 'label': preds})
my_submission.to_csv('submission.csv', index=False) 

In [ ]:
# Submission file ouput
print("Submission File: \n---------------\n")
print(my_submission.head()) # Predicted Output

In [ ]:
import os
os.chdir(r'/kaggle/working')

from IPython.display import FileLink 
FileLink(r'ResNet152.h5')